# Week 47

In [2]:
try:
    import fysisk_biokemi
    print("Already installed")
except ImportError:
    %pip install -q "fysisk_biokemi[colab] @ git+https://github.com/au-mbg/fysisk-biokemi.git"

------------------------------------------------------------------------

## Qualitative description of Michaelis-Menten kinetics

In [3]:
import fysisk_biokemi.widgets as fbw
from fysisk_biokemi.widgets.utils import enable_custom_widget_colab, disable_custom_widget_colab

import plotly
from IPython.display import display, HTML
enable_custom_widget_colab()
fbw.michaelis_menten_demo()

Use the widget created by the cell above to answer the following the
questions.

#### (a)

Setting $V_\mathrm{max} = 400$ and $[E]_\mathrm{tot} = 0.0005$, change
$K_M$ from 0.5 to 5 to 10 to 50. Describe the change in curve appearance
and explain it using the Michaelis-Menten (MM) equation.

#### (b)

What is the biological meaning of $K_M$?

#### (c)

Now set $[E]_\mathrm{tot} = 0.0005$, and $K_M = 5$, change
$V_\mathrm{max}$ from 40 to 100 to 900. Do you observe any changes?
Explain your answer using the widget and the MM equation

#### (d)

Now set $K_M = 5$ and $V_\mathrm{max} = 100$, change $[E]_\mathrm{tot}$
from 0.0005 to 0.05. Do you observe any change in curve appearance, in
kcat or in kcat/KM? Explain your answer

------------------------------------------------------------------------

## Enzyme inhibitors (I)

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
pd.set_option('display.max_rows', 6)

An enzyme obeying the Michaelis-Menten kinetics model was tested for
substrate conversion in the absence and presence of an inhibitor, called
inhibitor1 at a concentration of
$[\textrm{I}] = 2.5 \cdot 10^{-3} \ \textrm{M}$. The data set is
contained in the file `week49_6_inhib.xlsx`. Using this data a
researcher wanted to determine the type of inhibition.

Start by loading the dataset

In [5]:
from fysisk_biokemi.widgets import DataUploader
from IPython.display import display 
uploader = DataUploader()
uploader.display()

Run the next cell **after** uploading the file

In [6]:
df = uploader.get_dataframe()
display(df)

#### (a) Convert units

Convert the concentrations of substrate and the initial velocities to
units given in M and $\mathrm{M}\cdot \mathrm{s}^{-1}$, respectively.

In [8]:
df['[S]_(M)'] = ...
df['V0_no_inhib_(M/s)'] = ...
... # This one you'll have to do all on your own.
display(df)

#### (b) Plot

Plot the initial velocities of both experiments as a function of
substrate concentration in one plot. Estimate $k_\mathrm{cat}$ and
$V_\mathrm{max}$ in the presence and absence of inhibitor from the plot.

In [10]:
fig, ax = plt.subplots(figsize=(6, 4))

... # Your code for plotting here
... # and here. 

ax.set_xlabel('[S] (M)')
ax.set_ylabel('$V_0$ (M/s)')
ax.legend()

------------------------------------------------------------------------

The researcher wanted to determine $V_\mathrm{max}$ and $K_M$ values for
both experiments in order to correctly conclude on the type of
inhibitor.

#### (c)

Determine $V_\mathrm{max}$ and $K_M$ by fitting.

Start writing the function to fit with

In [12]:
def ...
    ...

And use the `curve_fit`-function to find the parameters

In [14]:
popt, _ = ...
vmax, km = popt

# With inhibitor
popt, _ = ...
vmax_inhib, km_inhib = popt

# Printing
print('Without inhibitor:')
print(f'\t V_max = {vmax:.2e}')
print(f'\t Km = {km:.2e}')
print('With inhibitor:')
print(f'\t V_max = {vmax_inhib:.2e}')
print(f'\t Km = {km_inhib:.2e}')

#### (d)

What type of inhibitor is inhibitor1?

#### (e)

Calculate $K_i$.  

------------------------------------------------------------------------

## Enzyme inhibitors (II)

In [16]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
pd.set_option('display.max_rows', 6)

Two enzyme inhibitors were identified as part of a drug discovery
program. To characterize the mechanism of action, the reaction kinetics
were analysed for the enzyme alone and in the presence of 5 $\mu$M of
each of the inhibitors as a function of substrate concentration
resulting in the data in the file `week49_7_inhib_II.xlsx`

Load the dataset

In [17]:
from fysisk_biokemi.widgets import DataUploader
from IPython.display import display 
uploader = DataUploader()
uploader.display()

Run the next cell **after** uploading the file

In [18]:
df = uploader.get_dataframe()
display(df)

#### (a) Convert & plot

Convert the measurements to SI units and plot the reaction rate against
time for all three reactions.

Start by converting the units

In [20]:
... # Your code to convert units here.

And then plot

In [22]:
fig, ax = plt.subplots()

col_names = ['enz_(M/s)', 'inhibitor2_(M/s)', 'inhibitor3_(M/s)']

for col_name in col_names:
    ax.plot(..., ..., 'o', label=col_name) # Replace ... with your code here. 
    
ax.legend()
ax.set_xlabel('Substrate concentration [M]', fontsize=14)
ax.set_ylabel('$V_0$', fontsize=14)
plt.show()

Based on the appearance of these plots:

-   Can you determine the type of inhibition?
-   Alternatively, can you exclude some mechanisms?

#### (b) Intuit parameters

Based on your answer to in (a) what do you expect for the parameters
$K_M$ and $V_\mathrm{max}$?

-   Which dataset would have the highest $V_\mathrm{max}$, which the
    lowest?
-   What about $K_M$?

#### (c) Fit

As always, we need the function we’re fitting with

In [24]:
def michaelis_menten(S, Vmax, Km):
    return (Vmax * S) / (S + Km)

Given we’re fitting on multiple datasets we will make our lives a bit
easier by making a little function

In [25]:
def fit_parameters(df, col_name):
    popt, _ = curve_fit(..., ..., ...) # Replace ... with your code.
    vmax, km = popt
    # Printing
    print(f'{col_name}:')
    print(f'\t V_max = {vmax:.2e}')
    print(f'\t Km = {km:.2e}')

In [27]:
for col_name in col_names:
    fit_parameters(df, col_name)

#### (d) Change

In the fits above you will likely see that all the fitted parameters
change between the different data sets – some by a little and some by a
lot. How big does a change have to be to be considered significant?

#### (e) $K_i$

Determine the $K_i$ for each of the two inhibitors.